# Bilingual Food Recognition and Nutrition Info Tool

This project is a user-friendly application that recognizes food items from images and provides detailed nutritional information in both English and Arabic. Built using Hugging Face's Visual Question Answering (VQA) model and Nutritionix's API, the tool offers a comprehensive analysis of various foods, including calories, protein, carbohydrates, fats, sugars, fiber, sodium, and serving size.

## Code

### Libraries

Start by installing and importing necessary libraries.

In [ ]:
# Install necessary libraries
!pip install transformers
!pip install gradio
!pip install wget

In [18]:
# Import necessary libraries
import gradio as gr
import wget
from transformers import pipeline
import requests

### API Setup

This cell sets up the connection to the Nutritionix API by defining the API URL and the required headers (App ID and App Key).

In [19]:
# Nutritionix API setup
api_url = "https://trackapi.nutritionix.com/v2/natural/nutrients"

# App ID, App Key provided by Nutritionix
headers = {
    "x-app-id": "dd773727",
    "x-app-key": "86f278fc4c7f276c386f280848acf3e6",
}

### Model Initiates

This cell loads the pre-trained models required for the project


In [20]:
# Load the Models

# Load the BLIP VQA Model (Recognize the food)
visual_quest_ans = pipeline("visual-question-answering", model="Salesforce/blip-vqa-base")

# Load the Translation Model (English to Arabic)
translation_eng_to_ar = pipeline("translation_en_to_ar", model="marefa-nlp/marefa-mt-en-ar")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


### Model Functions

This cell contains all the essential functions used in the project.

In [34]:
# Function to recognize food from the image using the VQA model
def food_recognizer(image):
    # Pass the image and the question to the model to identify the food on the image
    result = visual_quest_ans(image=image, question="What is the food or the drink in the image?")
    return result[0]['answer']

# Function to fetch nutritional information from Nutritionix API
def nutrition_info(food):
    # Prepare the data for the API request
    data = {
        "query": food,
        "timezone": "US/Eastern"
    }

    # Send a POST request to the Nutritionix API with the food item
    response = requests.post(api_url, headers=headers, json=data)

    # Get the nutritional information in JSON format
    nutritions = response.json()
    return nutritions

# Function to translate text from English to Arabic with preprocessing
def translator(text):
    text = text.strip()  # Remove leading/trailing spaces
    result = translation_eng_to_ar(text) # Use the translation model to translate the text
    result = result[0]['translation_text']
    return result

# Function to process food recognition and get nutrition info
def process_food_result(image, language):
    # Recognize the food item in the uploaded image
    food_item = food_recognizer(image)

    # Fetch nutritional information for the recognized food item
    nutritions_info = nutrition_info(food_item)

    # Extract nutritional information
    food_info = nutritions_info['foods'][0]
    calories = food_info['nf_calories']
    protein = food_info['nf_protein']
    carbs = food_info['nf_total_carbohydrate']
    fat = food_info['nf_total_fat']
    # Use 'Unknown' if value is not available
    sugars = food_info.get('nf_sugars', 'Unknown')
    fiber = food_info.get('nf_dietary_fiber', 'Unknown')
    sodium = food_info.get('nf_sodium', 'Unknown')
    serving_size = food_info.get('serving_weight_grams', 'Unknown')

    # Identify if the food item is a liquid (simple check for common drink categories)
    liquid_keywords = ['juice', 'water', 'milk', 'soda', 'tea', 'coffee']
    is_liquid = any(keyword in food_item.lower() for keyword in liquid_keywords)

    # Convert serving size to milliliters if it's a liquid
    if is_liquid and serving_size != 'Unknown':
        serving_size_ml = serving_size  # Assume 1 gram ≈ 1 milliliter for liquids
        serving_size_text_en = f"{serving_size_ml} mL"
        serving_size_text_ar = f"{serving_size_ml} مل"
    else:
        serving_size_text_en = f"{serving_size} grams"
        serving_size_text_ar = f"{serving_size} جرام"

    # Generate output in the selected language
    if language == "Arabic":
        # Translate the food item name to Arabic
        food_item_ar = translator(food_item)
        output_ar = f"""
        <div style='direction: rtl; text-align: right;'>
            <b>الطعام</b>: {food_item_ar}<br>
            <b>حجم الحصة</b>: {serving_size_text_ar}<br>
            <b>السعرات الحرارية</b>: {calories} كيلو كالوري<br>
            <b>البروتين</b>: {protein} جرام<br>
            <b>الكربوهيدرات</b>: {carbs} جرام<br>
            <b>السكر</b>: {sugars} جرام<br>
            <b>الألياف</b>: {fiber} جرام<br>
            <b>الصوديوم</b>: {sodium} مجم<br>
            <b>الدهون</b>: {fat} جرام
        </div>
        """
        return output_ar
    else:
       # For English output
        output_en = f"""
        <div style='text-align: left;'>
            <b>Food</b>: {food_item}<br>
            <b>Serving Size</b>: {serving_size_text_en}<br>
            <b>Calories</b>: {calories} kcal<br>
            <b>Protein</b>: {protein}g<br>
            <b>Carbohydrates</b>: {carbs}g<br>
            <b>Sugars</b>: {sugars}g<br>
            <b>Fiber</b>: {fiber}g<br>
            <b>Sodium</b>: {sodium}mg<br>
            <b>Fat</b>: {fat}g
        </div>
        """
        return output_en


# Gradio interface function
def gradio_function(image, language):
    # Call the process_food_result function to get the output
    result = process_food_result(image, language)
    return result

### Example Images

This cell downloads example images using the wget library.

In [30]:
# Define URLs of example images
image_urls = [
    "https://raw.githubusercontent.com/Abdulrahman078/ML_Datasets-Imgs-Vids/main/close-up-delicious-pizza.jpg",
    "https://raw.githubusercontent.com/Abdulrahman078/ML_Datasets-Imgs-Vids/main/assorted-desserts-with-chocolate-frosted-pink-glazed-sprinkles.jpg",
    "https://raw.githubusercontent.com/Abdulrahman078/ML_Datasets-Imgs-Vids/main/fried-fish-with-cranberries-wooden-board.jpg",
    "https://raw.githubusercontent.com/Abdulrahman078/ML_Datasets-Imgs-Vids/main/glass-water.jpg"
]

# Download the images and use their paths
example_images = [wget.download(url) for url in image_urls]


### Gradio Setup

This cell sets up the Gradio interface for the project.

In [35]:
# Setup the Gradio interface
iface = gr.Interface(
    fn=gradio_function, # Function to call
    inputs=[gr.Image(type="pil", label="Upload an image"), # Input: Image (in PIL format)
            gr.Dropdown(choices=["Arabic", "English"], label="Select Language", value="Arabic")], # Input: Dropdown for language selection
    outputs=gr.HTML(label="Food and Nutrition Information"), # Output: HTML for displaying nutrition info
    title="Bilingual Food Recognition and Nutrition Info Tool", # Title of the Gradio interface
    description="Upload an image of food, and the tool will recognize it and provide nutritional information in both English or Arabic languages.", # Description of the tool
    examples=[[img] for img in example_images]  # Add examples with the image and language
)

# Launch the Gradio interface with debug mode enabled
iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f814e829f1b5a38cbb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f814e829f1b5a38cbb.gradio.live
